In [20]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [21]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [23]:
# 2. Create a temporary view of the DataFrame.
# https://sparkbyexamples.com/spark/spark-createorreplacetempview-explained/#:~:text=So%20if%20you%20are%20comfortable,is%20not%20materialized%20into%20files.
df.createOrReplaceTempView("home_sales")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [24]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query1 = spark.sql("SELECT date, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY date ORDER BY date")
query1.show()

+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-01-01| 365088.0|
|2019-01-02|271644.29|
|2019-01-03|381218.86|
|2019-01-04|359586.14|
|2019-01-05|323683.57|
|2019-01-06|301091.13|
|2019-01-07| 327474.5|
|2019-01-08|304567.83|
|2019-01-09| 243619.0|
|2019-01-10| 287844.4|
|2019-01-11|277574.29|
|2019-01-12| 218507.0|
|2019-01-13|294603.86|
|2019-01-14|284434.57|
|2019-01-15| 254288.5|
|2019-01-16| 335106.5|
|2019-01-17|333439.09|
|2019-01-18|297542.56|
|2019-01-19|294484.22|
|2019-01-20|312395.33|
+----------+---------+
only showing top 20 rows



In [25]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query2 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
query2.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [26]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query3 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
query3.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [27]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
query4.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.6219983100891113 seconds ---


In [28]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [30]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query5 = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view")
query5.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|  10|401868.43|
| 100|1026669.5|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
|   2|397389.25|
|  20|399522.81|
|  21|399758.88|
|  22|402022.68|
|  23|403411.92|
|  24|400284.92|
|  25|401298.69|
+----+---------+
only showing top 20 rows

--- 0.5809981822967529 seconds ---


The uncached runtime was about 1.09 seconds, and the cached runtime was about 0.29 second. 

In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")

Py4JJavaError: An error occurred while calling o105.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputCom

In [28]:
# 11. Read the formatted parquet data.
p_df_p = spark.read.parquet('date_built_partitioned')

Py4JJavaError: An error occurred while calling o122.parquet.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:225)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$map$1(T

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_date_built_p')

NameError: name 'p_df_p' is not defined

In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

query6 = spark.sql("SELECT view, ROUND(AVG(price), 2) AS avg_price FROM p_date_built_p WHERE price >= 350000 GROUP BY view ORDER BY view")
query6.show()

print("--- %s seconds ---" % (time.time() - start_time))

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `p_date_built_p` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 52;
'Sort ['view ASC NULLS FIRST], true
+- 'Aggregate ['view], ['view, 'ROUND('AVG('price), 2) AS avg_price#584]
   +- 'Filter ('price >= 350000)
      +- 'UnresolvedRelation [p_date_built_p], [], false


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False